In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''
# S2S and C3S Model Regrid

# - Loads in all daily forecasts of sea ice extent
# - Regrids to polar stereographic,
# - Saves to netcdf files grouped by year

'\n\nThis code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. \nIf you use this code for a publication or presentation, please cite the reference in the README.md on the\nmain page (https://github.com/NicWayand/ESIO). \n\nQuestions or comments should be addressed to nicway@uw.edu\n\nCopyright (c) 2018 Nic Wayand\n\nGNU General Public License v3.0\n\n\n'

In [2]:

# Standard Imports
%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import scipy
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import os
import re
import glob
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import dask

# ESIO Imports
from esio import EsioData as ed
from esio import import_data

# must use the pygrib environment to unpack these grib files by hand. 
import pygrib  # must source activate pygrib
# https://jswhit.github.io/pygrib/docs/

The S2S sea ice reforecast files with perturbations (ensembles) are broken. I think they are missing a proper ensemble dimension. The standard Regrid_S2S_Model.py only reads in one ensemble member. Here have used a more primitive reader that reads grib files by record and figures out the dimensions by hand. Using the pygrib package and hence the pygrib environment

In [3]:
#FYI
import sys
print(sys.path)
print(sys.prefix)

['/home/disk/sipn/bitz/python/ESIO', '/home/disk/sipn/bitz/python/ESIO/notebooks', '/home/disk/sipn/nicway/anaconda3/envs/pygrib/lib/python36.zip', '/home/disk/sipn/nicway/anaconda3/envs/pygrib/lib/python3.6', '/home/disk/sipn/nicway/anaconda3/envs/pygrib/lib/python3.6/lib-dynload', '', '/home/disk/eos11/bitz/.local/lib/python3.6/site-packages', '/home/disk/sipn/nicway/anaconda3/envs/pygrib/lib/python3.6/site-packages', '/home/disk/sipn/nicway/anaconda3/envs/pygrib/lib/python3.6/site-packages/IPython/extensions', '/home/disk/eos11/bitz/.ipython']
/home/disk/sipn/nicway/anaconda3/envs/pygrib


In [9]:
E = ed.EsioData.load()

all_models = ['ncep', 'ukmo', 'cma', 'ecmwf','metreofr'] 
#all_models = ['metreofr']
#all_models = ['cma']
#all_models = ['ncep']
#all_models = ['ecmwf']


runType='reforecast'

var_dic = {}   # number of forecast times and ensemble members in each model
var_dic['ncep'] = {'NFT':43, 'NENS':3}
var_dic['cma'] = {'NFT':60, 'NENS':3}
var_dic['metreofr'] = {'NFT':60, 'NENS':14}
var_dic['ecmwf'] = {'NFT':46, 'NENS':10}
var_dic['ukmo'] = {'NFT':60, 'NENS':6}


stero_grid_file = E.obs['NSIDC_0051']['grid']

obs_grid = import_data.load_grid_info(stero_grid_file, model='NSIDC')
# Ensure latitude is within bounds (-90 to 90)
# Have to do this because grid file has 90.000001
obs_grid['lat_b'] = obs_grid.lat_b.where(obs_grid.lat_b < 90, other = 90)

# Regridding Options
method='nearest_s2d' # ['bilinear', 'conservative', 'nearest_s2d', 'nearest_d2s', 'patch']


In [10]:
Exploring = False
if Exploring:
    # trying to understand the data, none of this is needed
    model='cma'
    data_dir = E.model[model][runType]['native']
    data_out = E.model[model][runType]['sipn_nc']
    model_grid_file = E.model[model]['grid']
    print(data_dir)

    all_files = glob.glob(os.path.join(data_dir, '*.grib'))
    cf=all_files[10]
    print(cf)
    grbs = pygrib.open(cf)

    grbs.seek(0) # sets pointer to 0
    print(grbs.tell())  # tells where pointer is
    grbs.seek(0,2) # sets pointer to end
    Ntotal=grbs.tell()
    print(Ntotal)

    grbs.seek(0) # sets pointer to 0
    grb = grbs.select(name='Sea ice area fraction') # 
    grb[Ntotal-1]

    print(grb[0].forecastTime,grb[1].forecastTime,grb[2].forecastTime,grb[3].forecastTime) 
    print(grb[0].dataDate)
    
    # used this to convince myself that sequential records are different by a tiny bit
    # as one would expect for an ensemble
    cs=plt.imshow(grb[0].values-grb[1].values,vmin=-0.15, vmax=0.15)
    plt.colorbar(cs)
    #plt.clim([-0.5 0.5])
    plt.show()

In [11]:
for model in all_models:
    data_dir = E.model[model][runType]['native']
    data_out = E.model[model][runType]['sipn_nc']
    model_grid_file = E.model[model]['grid']

    #  number of forecast times and ensemble members for this model
    NFT=var_dic[model]['NFT']
    NENS=var_dic[model]['NENS']

    all_files = glob.glob(os.path.join(data_dir, '*.grib'))
    print("Found ",len(all_files)," files.")
    
    print("Only processing files not in the sipn_nc directory")

    # Remove any "empty" files (sometimes happends with ecmwf downloads)
    all_files_new = []
    for cf in all_files:
        if os.stat(cf).st_size > 0:
            all_files_new.append(cf)
        else:
            print("Found empty file: ",cf,". Consider deleting or redownloading.")
    all_files = sorted(all_files_new) # Replace and sort

    weights_flag = False 

    for cf in all_files:

        if (model in ['ecmwf','ukmo','metreofr']):
            f_out = os.path.join(data_out, os.path.basename(cf).split('.')[0]+'_Stereo.nc') # netcdf file out 
        else:
            f_out = os.path.join(data_out, os.path.basename(cf).split('.')[0]+'_b_Stereo.nc') # netcdf file out 

        print(f_out)
        if (os.path.isfile(f_out)):
        #print("Skipping ", os.path.basename(cf), " already imported.")
            continue # Skip, file already imported

        print(cf)
        grbs = pygrib.open(cf)
        grb = grbs.select(name='Sea ice area fraction')

        lat, lon= grb[0].latlons()
        lat=lat[:,0]
        lon=lon[0,:]
        
        da_ic=[]
        da_all=[]
        da_ft=[]
        for [count, igrb] in enumerate(grb):
            #print(igrb)
            ft = igrb.forecastTime
            ensemble = (count % NENS)
            ds=str(igrb.dataDate)  # init_time as str

            #print(count, ensemble, ft)
            da=xr.DataArray(igrb.values, coords=[('lat',lat),('lon',lon)])
            da.coords['fore_time'] = np.timedelta64(int(ft), 'h') #igrb.forecastTime
            da.coords['init_time'] = np.datetime64(datetime.datetime(int(ds[0:4]), int(ds[4:6]), int(ds[6:9]), 0, 0))
            da.coords['ensemble'] = np.int32(ensemble) #ensemble
            da = da.expand_dims('fore_time')
            da = da.expand_dims('init_time')
            da = da.expand_dims('ensemble')

            da_all.append(da)

            if ensemble==NENS-1:
                da_all = xr.concat(da_all, dim='ensemble')
                da_ft.append(da_all)
                da_all = []

                if ft==grb[NFT*NENS-1].forecastTime:
                    da_ft=xr.concat(da_ft, dim='fore_time')
                    da_ic.append(da_ft)
                    da_ft=[]
                    #print('Finished pass through FT and ENS ',count, ensemble,ft)


        da_ic=xr.concat(da_ic, dim='init_time')
        da_ic.name = 'sic'
        grbs.close()

        #print(da_ic)

        # Calculate regridding matrix
        regridder = xe.Regridder(da_ic, obs_grid, method, periodic=True, reuse_weights=weights_flag)
        weights_flag = True # Set true for following loops

        # Add NaNs to empty rows of matrix (forces any target cell with ANY source cells containing NaN to be NaN)
        if method=='conservative':
            print('Removing edge cells that contain source NaN cells, should probably check here')
            regridder = import_data.add_matrix_NaNs(regridder)

        # Regrid variable
        var_out = regridder(da_ic)

        # Expand dims
        var_out = import_data.expand_to_sipn_dims(var_out)


        if (model in ['ecmwf','ukmo','metreofr']):
            f_out = os.path.join(data_out, os.path.basename(cf).split('.')[0]+'_Stereo.nc') # netcdf file out 
            var_out.isel(init_time=slice(0,16)).to_netcdf(f_out)
        else:

            f_out = os.path.join(data_out, os.path.basename(cf).split('.')[0]+'_a_Stereo.nc') # netcdf file out 
            var_out.isel(init_time=slice(0,16)).to_netcdf(f_out)

            f_out = os.path.join(data_out, os.path.basename(cf).split('.')[0]+'_b_Stereo.nc') # netcdf file out 
            var_out.isel(init_time=slice(16,33)).to_netcdf(f_out)

        print('Saved ', f_out)


Found  144  files.
Only processing files not in the sipn_nc directory
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_01_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_02_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_03_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_04_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_05_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_06_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_07_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_08_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_09_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_10_b_Stereo.nc
/home/disk/sipn/nicway/data/model/ncep/reforecast/sipn_nc/ncep_1999_11_b_Stereo.nc
/home/disk/sipn/n

Found  1008  files.
Only processing files not in the sipn_nc directory
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_01_01_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_01_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_01_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_01_25_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_02_01_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_02_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_02_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_02_25_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_03_01_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_03_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_1995_03_17_Stereo.nc
/home

/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_02_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_02_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_02_25_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_03_01_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_03_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_03_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_03_25_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_04_01_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_04_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_04_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_04_25_Stereo.nc
/home/disk/sipn/nicway/data/model/ukmo/reforecast/sipn_nc/ukmo_2003_05_01_St

/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_01_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_02_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_03_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_04_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_05_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_06_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_07_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_08_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_09_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_10_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_11_b_Stereo.nc
/home/disk/sipn/nicway/data/model/cma/reforecast/sipn_nc/cma_1995_12_b_Stereo.nc
/home/disk/sipn/nicway/data/

Found  2080  files.
Only processing files not in the sipn_nc directory
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_06_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_09_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_13_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_16_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_20_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_23_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_27_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_08_30_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_09_03_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998_09_06_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_1998

/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_03_18_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_03_21_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_03_25_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_03_28_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_01_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_04_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_08_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_11_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_15_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_18_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2002_04_22_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sip

/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_08_23_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_08_27_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_08_30_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_03_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_06_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_10_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_13_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_20_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_24_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2005_09_27_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sip

/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_08_16_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_08_20_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_08_23_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_08_27_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_08_30_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_09_03_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_09_06_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_09_10_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_09_13_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_09_17_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sipn_nc/ecmwf_2010_09_20_Stereo.nc
/home/disk/sipn/nicway/data/model/ecmwf/reforecast/sip

Found  1056  files.
Only processing files not in the sipn_nc directory
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_01_01_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_01_08_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_01_15_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_01_22_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_02_01_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_02_08_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_02_15_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_02_22_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_03_01_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_1993_03_08_Stereo.nc
/home/dis

/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_03_08_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_03_15_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_03_22_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_04_01_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_04_08_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_04_15_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_04_22_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_05_01_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_05_08_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_05_15_Stereo.nc
/home/disk/sipn/nicway/data/model/metreofr/reforecast/sipn_nc/metreofr_2000_05_2